In [1]:
import pandas as pd

# Recommender System:

- The last thing to do is to use our saved models to recommend items to users:



### For the requested user:
- Calculate the score for every item.
- Sort the items based on the score and output the top results.


### Check which users exist on the test set

In [20]:
import pickle
import pandas as pd
import numpy as np

import os

#Keras
from keras.models import load_model
from keras import backend as K

# Tensorflow
import tensorflow as tf

from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


### Set and Check GPUs

In [21]:
def set_check_gpu():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.per_process_gpu_memory_fraction =1 # allow all of the GPU memory to be allocated
    # for 8 GPUs
    # cfg.gpu_options.visible_device_list = "0,1,2,3,4,5,6,7" # "0,1"
    # for 1 GPU
    cfg.gpu_options.visible_device_list = "0"
    #cfg.gpu_options.allow_growth = True  # # Don't pre-allocate memory; dynamically allocate the memory used on the GPU as-needed
    #cfg.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = K.tf.Session(config=cfg)
    K.set_session(sess)  # set this TensorFlow session as the default session for Keras

    print("* TF version: ", [tf.__version__, tf.test.is_gpu_available()])
    print("* List of GPU(s): ", tf.config.experimental.list_physical_devices() )
    print("* Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU'))) 
  
    
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
    # set for 8 GPUs
#     os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7";
    # set for 1 GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "0";

    # Tf debugging option
    tf.debugging.set_log_device_placement(True)

    gpus = tf.config.experimental.list_physical_devices('GPU')

    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

#     print(tf.config.list_logical_devices('GPU'))
    print(tf.config.experimental.list_physical_devices('GPU'))
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [22]:
set_check_gpu()

* TF version:  ['1.15.2', True]
* List of GPU(s):  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
* Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [2]:
all_info = pd.read_parquet("./data/Cleaned_meta_Clothing_Shoes_and_Jewelry.parquet")
ratings_test = pd.read_parquet('./data/ratings_test.parquet')
ratings_train = pd.read_parquet('./data/ratings_train.parquet')

In [3]:
ratings_test.user_id.value_counts().sort_values(ascending=False).head(10)

635       28
33102     26
1123      25
136939    24
7401      23
136131    22
5768      20
34951     20
52789     20
4959      20
Name: user_id, dtype: int64

### ** Create a  recommendation example dataset of 100 users from the test set and 100 random items for each and predict recommendations for them

In [4]:
items = all_info.asin.unique()
item_map = {i:val for i,val in enumerate(items)}
inverse_item_map = {val:i for i,val in enumerate(items)}
all_info["old_item_id"] = all_info["asin"] # copying for join with metadata
all_info["item_id"] = all_info["asin"].map(inverse_item_map)
items = all_info.item_id.unique()
print ("We have %d unique items in metadata "%items.shape[0])

We have 2681355 unique items in metadata 


In [5]:
all_info['description'] = all_info['description'].fillna(all_info['title'].fillna('no_data'))
all_info['title'] = all_info['title'].fillna(all_info['description'].fillna('no_data').apply(str).str[:20])
all_info['image'] = all_info['image'].fillna('no_data')
all_info['price'] = pd.to_numeric(all_info['price'],errors="coerce")
all_info['price'] = all_info['price'].fillna(all_info['price'].median()) 

In [6]:
# creating metadata mappings 
titles = all_info['title'].unique()
titles_map = {i:val for i,val in enumerate(titles)}
inverse_titles_map = {val:i for i,val in enumerate(titles)}

price = all_info['price'].unique()
price_map = {i:val for i,val in enumerate(price)}
inverse_price_map = {val:i for i,val in enumerate(price)}

print ("We have %d prices" %price.shape)
print ("We have %d titles" %titles.shape)


all_info['price_id'] = all_info['price'].map(inverse_price_map)
all_info['title_id'] = all_info['title'].map(inverse_titles_map)

# creating dict from 
item2prices = {}
for val in all_info[['item_id','price_id']].dropna().drop_duplicates().iterrows():
    item2prices[val[1]["item_id"]] = val[1]["price_id"]

item2titles = {}
for val in all_info[['item_id','title_id']].dropna().drop_duplicates().iterrows():
    item2titles[val[1]["item_id"]] = val[1]["title_id"]
        


# populating the rating dataset with item metadata info
ratings_train["price_id"] = ratings_train["item_id"].map(lambda x : item2prices[x])
ratings_train["title_id"] = ratings_train["item_id"].map(lambda x : item2titles[x])


# populating the test dataset with item metadata info
ratings_test["price_id"] = ratings_test["item_id"].map(lambda x : item2prices[x])
ratings_test["title_id"] = ratings_test["item_id"].map(lambda x : item2titles[x])


We have 1 prices
We have 2507255 titles


In [ ]:
# ratings_test = pd.read_parquet('./data/ratings_test.parquet')
# ratings_train = pd.read_parquet('./data/ratings_train.parquet')

In [10]:
items = all_info.item_id.unique()
df_items = pd.DataFrame(data=items.flatten(),columns=['item_id'])
df_items = pd.merge(df_items,all_info,how='left',left_on=('item_id'),right_on=('item_id'))
df_items= df_items.sample(100)
df_items['key'] = 1
print ("We have %d unique items  "%df_items['item_id'].shape[0])
# df_items= df_items[['item_id', 'description', 'category', 'title', 'title_id', 'price', 'price_id', 'brand', 'key']]
df_items= df_items[['item_id', 'description', 'category', 'title', 'title_id', 'price', 'price_id', 'key']]
print(df_items.shape)
df_items.head(2)

We have 100 unique items  
(100, 8)


,item_id,description,category,title,title_id,price,price_id,key
923406,919702,Aokin Sexy Ladies Women Push up Padded Bikini ...,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",Aokin Sexy Ladies Women Push up Padded Bikini ...,871016,NaN,0,1
1092135,1088431,Baby Phat Women's Chance Round Toe Platform Pu...,"[Clothing, Shoes & Jewelry, Women, Shoes, Pumps]",Baby Phat Women's Chance Round Toe Platform Pu...,1029726,NaN,0,1


In [11]:
users = ratings_test.user_id.unique()
df_users = pd.DataFrame(data=users.flatten(),columns=['user_id'])
df_users = pd.merge(df_users,ratings_test,how='left',left_on=('user_id'),right_on=('user_id'))
df_users= df_users.sample(100)
df_users['key'] = 1
print ("We have %d unique users  "%df_users['user_id'].shape[0])
df_users= df_users[['user_id', 'user_name', 'key']]
print(df_users.shape)
df_users.head(2)

We have 100 unique users  
(100, 3)


,user_id,user_name,key
2644,5053,michelle,1
171483,297377,Amazon Customer,1


## Merge users and item and items metadata

In [16]:
df_unseenData= pd.merge(df_users, df_items, on='key')
del df_unseenData['key']
print ("We have %d unique records in the recommendation example dataset "%df_unseenData.shape[0])
print(df_unseenData.shape)
df_unseenData.sample(10)

We have 10000 unique records in the recommendation example dataset 
(10000, 9)


,user_id,user_name,item_id,description,category,title,title_id,price,price_id
7251,86155,M. Lee Taylor,33378,[From the Phantom of the Opera Collection. Pha...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ea...",Fun To Collect Phantom of the Opera Phantom Ma...,31655,NaN,0
4520,163736,Chris Uppercut,1226645,[A beautiful handbag with a grey blue color tone],"[Clothing, Shoes & Jewelry, Women, Handbags & ...",Lauren Ferretti Women's - Fashion Grey Shoulde...,1158125,NaN,0
7984,36849,Andy L,2410280,Gerber Onesies Baby Boy Sleep N' Play Sleepers...,"[Clothing, Shoes & Jewelry, Baby]",Gerber Onesies Baby Boy Sleep N' Play Sleepers...,2254698,NaN,0
1158,327960,Mommytsunami,190572,[This fashionable and lightweight purse hooks ...,"[Clothing, Shoes & Jewelry, Women, Accessories...",Pink Diamond Folding Handbag Purse Hook Hanger,181318,NaN,0
3052,62758,Ian C,1198015,[100% polyester jersey knit; heat transfer lab...,"[Clothing, Shoes & Jewelry, Men, Clothing, Act...",Gildan G420 Performance 4.5 oz. T-Shirt,722013,NaN,0
107,297377,Amazon Customer,789144,[In 1920 Sbicca Shoes was launched by Frank an...,"[Clothing, Shoes & Jewelry, Women, Shoes, Sand...",Sbicca Women's Kismet,747991,NaN,0
3560,257633,HappyMama89,39301,[These Bates 1728 Women's 8-Inch DuraShocks St...,"[Clothing, Shoes & Jewelry, Women, Uniforms, W...","Bates Women's 8"" Durashocks Steel Toe Boot",37173,NaN,0
6931,67469,Alana,76856,[A classic polo crafted of soft pima cotton. A...,"[Clothing, Shoes & Jewelry, Men, Clothing, Shi...",Nautica Men's SS Clarkson Stripe Polo,72720,NaN,0
3767,44308,Rachel N.,903763,"[, The Amazon Curated Collection, Discover the...","[Clothing, Shoes & Jewelry, Women, Jewelry, Ea...",Sterling Silver Genuine White Topaz and Blue S...,856256,NaN,0
1861,472996,Terry,418294,"[, The Amazon Curated Collection, Discover the...","[Clothing, Shoes & Jewelry, Women, Jewelry, Br...",Sterling Silver Quartz Floral 9mm Freshwater C...,397678,NaN,0


In [17]:
df_unseenData.columns

Index(['user_id', 'user_name', 'item_id', 'description', 'category', 'title',
       'title_id', 'price', 'price_id'],
      dtype='object')

In [18]:
from os import listdir
from os.path import isfile, join

mypath = './models'

onlyfiles = [f.replace('.h5', '') for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['dense_2_Concatenate_10_embeddings_10_epochs',
 'dense_1_Multiply_50_embeddings_7_epochs',
 'dense_2_Concatenate_20_embeddings_25_epochs',
 'dense_1_Multiply_50_embeddings_4_epochs_dropout',
 'dense_2_Meta_Concatenate_15_embeddings_50_epochs-DropOut0.6',
 'matrix_facto_10_embeddings_3_epochs',
 'dense_1_Multiply_50_embeddings_20_epochs_dropout',
 'dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.6-dropout-input',
 'dense_1_Multiply_50_embeddings_7_epochs_dropout',
 'dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.8',
 'matrix_facto_10_embeddings_100_epochs',
 'dense_2_Meta_Concatenate_15_embeddings_30_epochs-DropOut0.6',
 'matrix_facto_10_embeddings_20_epochs',
 'dense_4_Multiply_5_embeddings_7_epochs',
 'dense_1_Multiply_50_embeddings_2_epochs_dropout']

## Predict the ratings for the items and users in the a recommendation example dataset:

In [23]:
load_path = "./models/"

models =['dense_2_Concatenate_10_embeddings_10_epochs',
 'dense_1_Multiply_50_embeddings_7_epochs',
 'dense_2_Concatenate_20_embeddings_25_epochs',
 'dense_1_Multiply_50_embeddings_4_epochs_dropout',
 'matrix_facto_10_embeddings_3_epochs',
 'dense_1_Multiply_50_embeddings_20_epochs_dropout',
 'dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.6-dropout-input',
 'dense_1_Multiply_50_embeddings_7_epochs_dropout',
 'dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.8',
 'matrix_facto_10_embeddings_100_epochs',
 'matrix_facto_10_embeddings_20_epochs',
 'dense_4_Multiply_5_embeddings_7_epochs',
 'dense_1_Multiply_50_embeddings_2_epochs_dropout']

models_meta = [
 'dense_2_Meta_Concatenate_15_embeddings_50_epochs-DropOut0.6',
 'dense_2_Meta_Concatenate_15_embeddings_30_epochs-DropOut0.6',
]

# for mod in models:
#     model = load_model(load_path+mod+'.h5')
#     df_unseenData['preds_' + mod] = model.predict([df_unseenData['user_id'],
#                                                    df_unseenData['item_id'],
#                                                    df_unseenData['price_id'],
#                                                    df_unseenData['title_id']])

for mod in models:
    model = load_model(load_path+mod+'.h5')
    df_unseenData['preds_' + mod] = model.predict([df_unseenData['user_id'],
                                                   df_unseenData['item_id']])


df_unseenData.head(2)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









,user_id,user_name,item_id,description,category,title,title_id,price,price_id,preds_dense_2_Concatenate_10_embeddings_10_epochs,...,preds_dense_1_Multiply_50_embeddings_4_epochs_dropout,preds_matrix_facto_10_embeddings_3_epochs,preds_dense_1_Multiply_50_embeddings_20_epochs_dropout,preds_dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.6-dropout-input,preds_dense_1_Multiply_50_embeddings_7_epochs_dropout,preds_dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.8,preds_matrix_facto_10_embeddings_100_epochs,preds_matrix_facto_10_embeddings_20_epochs,preds_dense_4_Multiply_5_embeddings_7_epochs,preds_dense_1_Multiply_50_embeddings_2_epochs_dropout
0,5053,michelle,919702,Aokin Sexy Ladies Women Push up Padded Bikini ...,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",Aokin Sexy Ladies Women Push up Padded Bikini ...,871016,NaN,0,4.014494,...,4.141841,0.0,4.311826,3.961529,3.757429,3.848483,0.0,0.0,4.292699,3.905577
1,5053,michelle,1088431,Baby Phat Women's Chance Round Toe Platform Pu...,"[Clothing, Shoes & Jewelry, Women, Shoes, Pumps]",Baby Phat Women's Chance Round Toe Platform Pu...,1029726,NaN,0,4.014494,...,4.141841,0.0,4.311826,3.961529,3.757429,3.848483,0.0,0.0,4.292699,3.905577


In [24]:
df_unseenData['user_id'].head(n=2)

0    5053
1    5053
Name: user_id, dtype: int64

In [25]:
df_unseenData.columns

Index(['user_id', 'user_name', 'item_id', 'description', 'category', 'title',
       'title_id', 'price', 'price_id',
       'preds_dense_2_Concatenate_10_embeddings_10_epochs',
       'preds_dense_1_Multiply_50_embeddings_7_epochs',
       'preds_dense_2_Concatenate_20_embeddings_25_epochs',
       'preds_dense_1_Multiply_50_embeddings_4_epochs_dropout',
       'preds_matrix_facto_10_embeddings_3_epochs',
       'preds_dense_1_Multiply_50_embeddings_20_epochs_dropout',
       'preds_dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.6-dropout-input',
       'preds_dense_1_Multiply_50_embeddings_7_epochs_dropout',
       'preds_dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.8',
       'preds_matrix_facto_10_embeddings_100_epochs',
       'preds_matrix_facto_10_embeddings_20_epochs',
       'preds_dense_4_Multiply_5_embeddings_7_epochs',
       'preds_dense_1_Multiply_50_embeddings_2_epochs_dropout'],
      dtype='object')

## Check which users exist on the example set

In [26]:
df_unseenData.user_id.value_counts().sort_values(ascending=False).head(5)
df_unseenData[['user_id','preds_dense_1_Multiply_50_embeddings_7_epochs']].sort_values('preds_dense_1_Multiply_50_embeddings_7_epochs',ascending=True).head(5)

,user_id,preds_dense_1_Multiply_50_embeddings_7_epochs
8128,36754,1.930570
3528,257633,2.270640
7528,70795,2.393019
1728,49526,2.499284
7028,50398,2.512622


## A function that will return recommendation list for a given user

In [27]:
df_unseenData.head(n=3)

,user_id,user_name,item_id,description,category,title,title_id,price,price_id,preds_dense_2_Concatenate_10_embeddings_10_epochs,...,preds_dense_1_Multiply_50_embeddings_4_epochs_dropout,preds_matrix_facto_10_embeddings_3_epochs,preds_dense_1_Multiply_50_embeddings_20_epochs_dropout,preds_dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.6-dropout-input,preds_dense_1_Multiply_50_embeddings_7_epochs_dropout,preds_dense_4_Multiply_5_embeddings_7_epochs-DropOut-0.8,preds_matrix_facto_10_embeddings_100_epochs,preds_matrix_facto_10_embeddings_20_epochs,preds_dense_4_Multiply_5_embeddings_7_epochs,preds_dense_1_Multiply_50_embeddings_2_epochs_dropout
0,5053,michelle,919702,Aokin Sexy Ladies Women Push up Padded Bikini ...,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",Aokin Sexy Ladies Women Push up Padded Bikini ...,871016,NaN,0,4.014494,...,4.141841,0.0,4.311826,3.961529,3.757429,3.848483,0.0,0.0,4.292699,3.905577
1,5053,michelle,1088431,Baby Phat Women's Chance Round Toe Platform Pu...,"[Clothing, Shoes & Jewelry, Women, Shoes, Pumps]",Baby Phat Women's Chance Round Toe Platform Pu...,1029726,NaN,0,4.014494,...,4.141841,0.0,4.311826,3.961529,3.757429,3.848483,0.0,0.0,4.292699,3.905577
2,5053,michelle,370649,[NIKE FLYKNIT RACER - 'OREO' -526628-012],"[Clothing, Shoes & Jewelry, Men, Shoes, Athlet...","NIKE Flyknit Racer - 'Oreo' -526628-012 Black,...",353019,NaN,0,4.014494,...,4.141841,0.0,4.311826,3.961529,3.757429,3.848483,0.0,0.0,4.292699,3.905577


In [28]:
load_path = "./models/"

def get_recommendations(userID , model_scr, df_Data):
    if userID not in df_Data['user_id'].values:
        print("\nUser ID not found %d" %userID)
        return userID
       
    print("\nRecommendations for user id %d Name: %s is:" % (userID, df_Data.loc[df_Data['user_id'] == userID, 'user_name'].values[0]))
   
    df_output=df_Data.loc[df_Data['user_id'] == userID][['item_id','title','description','category','price', model_scr,    
     ]].sort_values(model_scr,ascending=False).set_index('item_id')
#     print(df_output)
    
    df_output.rename(columns={model_scr: 'score'}, inplace=True)
    
    #add ASIN form item_id
    df_output['asin'] = df_Data['item_id'].apply(item_map)
    
    return df_output

### Recommend items to a given user

- Using dense_1_Multiply_50_embeddings_7_epochs

In [30]:
#######  User ID: 36754                       
df_output = get_recommendations(userID=36754 ,model_scr='preds_dense_1_Multiply_50_embeddings_7_epochs', df_Data=df_unseenData)
df_output.head(10)



Recommendations for user id 36754 Name: MGH is:


,title,description,category,price,score
item_id,,,,,
76856,Nautica Men's SS Clarkson Stripe Polo,[A classic polo crafted of soft pima cotton. A...,"[Clothing, Shoes & Jewelry, Men, Clothing, Shi...",NaN,3.157357
127697,Santa Suit - Men's 8 piece Santa Costume (XL (...,[This Santa Suit has it all! It includes the P...,"[Clothing, Shoes & Jewelry, Costumes & Accesso...",NaN,3.059101
33378,Fun To Collect Phantom of the Opera Phantom Ma...,[From the Phantom of the Opera Collection. Pha...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ea...",NaN,3.011844
190572,Pink Diamond Folding Handbag Purse Hook Hanger,[This fashionable and lightweight purse hooks ...,"[Clothing, Shoes & Jewelry, Women, Accessories...",NaN,2.936568
75478,Caravelle by Bulova Men's 44D002 Diamond Accen...,"[In 1875, a young Czech immigrant named Joseph...","[Clothing, Shoes & Jewelry, Men, Watches, Wris...",NaN,2.893246
39301,"Bates Women's 8"" Durashocks Steel Toe Boot",[These Bates 1728 Women's 8-Inch DuraShocks St...,"[Clothing, Shoes & Jewelry, Women, Uniforms, W...",NaN,2.785911
242039,Light Blue Sparkle Captive Ring Eyebrow Tragus...,Light Blue Sparkle Captive Ring Eyebrow Tragus...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Bo...",NaN,2.775016
42351,14k White Gold Pear Shaped Aquamarine and Diam...,14k White Gold Pear Shaped Aquamarine and Diam...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ne...",NaN,2.754715
241891,Black 3mm 14G Internally Threaded Replacement ...,Black 3mm 14G Internally Threaded Replacement ...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Bo...",NaN,2.749818


## Make predictions for another user using another model:

In [85]:
#######  User ID

user_id = 257633
df_output = get_recommendations(userID=user_id ,model_scr='preds_dense_1_Multiply_50_embeddings_7_epochs',df_Data=df_unseenData)
df_output.head(10)


Recommendations for user id 257633 Name: HappyMama89 is:


,title,description,category,price,score
item_id,,,,,
76856,Nautica Men's SS Clarkson Stripe Polo,[A classic polo crafted of soft pima cotton. A...,"[Clothing, Shoes & Jewelry, Men, Clothing, Shi...",NaN,3.406863
127697,Santa Suit - Men's 8 piece Santa Costume (XL (...,[This Santa Suit has it all! It includes the P...,"[Clothing, Shoes & Jewelry, Costumes & Accesso...",NaN,3.380237
33378,Fun To Collect Phantom of the Opera Phantom Ma...,[From the Phantom of the Opera Collection. Pha...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ea...",NaN,3.332020
190572,Pink Diamond Folding Handbag Purse Hook Hanger,[This fashionable and lightweight purse hooks ...,"[Clothing, Shoes & Jewelry, Women, Accessories...",NaN,3.309201
75478,Caravelle by Bulova Men's 44D002 Diamond Accen...,"[In 1875, a young Czech immigrant named Joseph...","[Clothing, Shoes & Jewelry, Men, Watches, Wris...",NaN,3.287312
39301,"Bates Women's 8"" Durashocks Steel Toe Boot",[These Bates 1728 Women's 8-Inch DuraShocks St...,"[Clothing, Shoes & Jewelry, Women, Uniforms, W...",NaN,3.249515
42351,14k White Gold Pear Shaped Aquamarine and Diam...,14k White Gold Pear Shaped Aquamarine and Diam...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ne...",NaN,3.223683
105850,Leg Avenue Women's Comic Book Hero,"[3 piece comic book hero, includes halter cat ...","[Clothing, Shoes & Jewelry, Costumes & Accesso...",NaN,3.221000
85350,Lovely People Women's Kimberly Flat,[Shoes for Lovely People was created by appare...,"[Clothing, Shoes & Jewelry, Women, Shoes, Flat...",NaN,3.213484


In [66]:
df_output = df_output.reset_index()
df_output['user_id'] = user_id
df_output['asin'] = df_output['item_id'].apply(lambda x : item_map[x])
df_output['url'] = df_output['item_id'].apply(lambda x : 'https://www.amazon.com/dp/'+item_map[x])
df_output = df_output[['user_id','item_id', 'score', 'asin', 'url', 'title', 'description', 'category', 'price']]

In [76]:
df_output.head()

,user_id,item_id,score,asin,url,title,description,category,price
0,257633,76856,3.406863,B00139CLCC,https://www.amazon.com/dp/B00139CLCC,Nautica Men's SS Clarkson Stripe Polo,[A classic polo crafted of soft pima cotton. A...,"[Clothing, Shoes & Jewelry, Men, Clothing, Shi...",NaN
1,257633,127697,3.380237,B001LQRDMA,https://www.amazon.com/dp/B001LQRDMA,Santa Suit - Men's 8 piece Santa Costume (XL (...,[This Santa Suit has it all! It includes the P...,"[Clothing, Shoes & Jewelry, Costumes & Accesso...",NaN
2,257633,33378,3.332020,B000K4SVCA,https://www.amazon.com/dp/B000K4SVCA,Fun To Collect Phantom of the Opera Phantom Ma...,[From the Phantom of the Opera Collection. Pha...,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ea...",NaN
3,257633,190572,3.309201,B002UZ1UW0,https://www.amazon.com/dp/B002UZ1UW0,Pink Diamond Folding Handbag Purse Hook Hanger,[This fashionable and lightweight purse hooks ...,"[Clothing, Shoes & Jewelry, Women, Accessories...",NaN
4,257633,75478,3.287312,B0012XSW68,https://www.amazon.com/dp/B0012XSW68,Caravelle by Bulova Men's 44D002 Diamond Accen...,"[In 1875, a young Czech immigrant named Joseph...","[Clothing, Shoes & Jewelry, Men, Watches, Wris...",NaN


### select 100 users from unseen data

In [86]:
all_predicted_df = pd.DataFrame()

for user_id in df_unseenData.sample(n=100, random_state=0)['user_id']:
    print("selected 100 user_id:", user_id)
    df_output = get_recommendations(userID=user_id ,model_scr='preds_dense_1_Multiply_50_embeddings_7_epochs',df_Data=df_unseenData)
    df_output = df_output.reset_index()
    df_output['user_id'] = user_id
    df_output['asin'] = df_output['item_id'].apply(lambda x : item_map[x])
    df_output['url'] = df_output['item_id'].apply(lambda x : 'https://www.amazon.com/dp/'+item_map[x])
    df_output = df_output[['user_id','item_id', 'score', 'asin', 'url', 'title', 'description', 'category', 'price']]
    
    #concat
    all_predicted_df = all_predicted_df.append(df_output)

selected 100 user_id: 113099

Recommendations for user id 113099 Name: Angela Travis is:
selected 100 user_id: 111238

Recommendations for user id 111238 Name: A. Webb is:
selected 100 user_id: 243987

Recommendations for user id 243987 Name: Emily is:
selected 100 user_id: 423151

Recommendations for user id 423151 Name: Stephen Sobell is:
selected 100 user_id: 243987

Recommendations for user id 243987 Name: Emily is:
selected 100 user_id: 180552

Recommendations for user id 180552 Name: Penny Gwinn is:
selected 100 user_id: 152976

Recommendations for user id 152976 Name: BakingRaver is:
selected 100 user_id: 75096

Recommendations for user id 75096 Name: Mari23avelar is:
selected 100 user_id: 199636

Recommendations for user id 199636 Name: Reinvented is:
selected 100 user_id: 183358

Recommendations for user id 183358 Name: Patricia L. McQueary is:
selected 100 user_id: 39965

Recommendations for user id 39965 Name: Mrs. Z is:
selected 100 user_id: 472996

Recommendations for user

In [93]:
all_predicted_df.to_csv('for_100_users_prediction.csv', header=True, index=False)